In [1]:
%load_ext autotime

In [2]:
# Importar librerias
import pandas as pd
import numpy as np

import zipfile
from datetime import date
from sklearn import metrics
from sklearn.metrics import accuracy_score


from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

from numpy.testing import assert_allclose

import tensorflow as tf

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adadelta

import os
from pathlib import Path


C:\Users\Ed\anaconda3\envs\tf-gpu\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Ed\anaconda3\envs\tf-gpu\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Ed\anaconda3\envs\tf-gpu\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Ed\anaconda3\envs\tf-gpu\lib\site-packages\tensor

time: 1min 4s


In [3]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

time: 2 ms


In [4]:
exp_name = '8'
folder = 'Resultados/' + exp_name
my_file = Path(folder)
if os.path.exists(my_file):
    print('already exists')
else:
    os.makedirs(folder)
    


already exists
time: 329 ms


In [5]:
ch_folder = folder + '/Checkpoints'
my_file = Path(ch_folder)
if os.path.exists(my_file):
    print('already exists')
else:
    os.makedirs(ch_folder)

already exists
time: 171 ms


### Read datasets

In [6]:
#Importar los datasets
url_embeddings_average_individual = zipfile.ZipFile('../Data/average_bigram_topics_sentiment.zip')

embeddings_average_individual = pd.read_csv(url_embeddings_average_individual.open('average_bigram_topics_sentiment.csv'), index_col = 0)

embeddings_average_individual['Date'] =  pd.to_datetime(embeddings_average_individual['Date'], format='%Y-%m-%d')
# embeddings_average_individual.sort_values('Date', inplace=True)
# embeddings_average_individual.reset_index(drop = True , inplace=True)
embeddings_average_individual.reset_index(inplace=True)
embeddings_average_individual.fillna(0, inplace=True)
embeddings_average_individual.tail(2)

,0,1,2,3,4,5,6,7,8,9,...,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,sentiment
49649,0.000830,0.050684,-0.078711,0.069385,-0.120630,0.095508,0.036621,0.002893,0.096631,0.027344,...,0.0222,0.0178,0.0149,0.0128,0.0113,0.01,0.1759,0.698,0.0,-0.1
49650,0.073608,0.090820,-0.069580,-0.088684,-0.126465,-0.135132,-0.096680,-0.334961,0.136230,0.150879,...,0.0141,0.0113,0.0000,0.1980,0.2274,0.00,0.0000,0.000,0.0,0.0


time: 6.03 s


### Embedding Promedio

In [7]:
# Selecciono la fecha para la cual hago el corte de train y test
training_end = pd.to_datetime("2013-12-31")
num_training = len(embeddings_average_individual[(embeddings_average_individual["Date"]) <= training_end])

time: 100 ms


In [8]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_average_individual[embeddings_average_individual['Date']<='2014-12-31']

time: 247 ms


In [9]:
# Se separa en train y test
x_train = data.drop(["Top","Label", "Date"], axis=1)[:num_training]
x_test = data.drop(["Top",'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]


x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(len(x_train), 1, x_train.shape[1])
reshape_x_train.shape

x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(len(x_test), 1, x_train.shape[1])
reshape_x_test.shape

(6294, 1, 311)

time: 286 ms


### Definir espacio de busqueda

In [10]:
space = {
    'units1': hp.choice('units1', [10, 64, 128, 256, 512]),
    'units2': hp.choice('units2', [10, 64, 128, 256, 512]),
                 
    'dropout1': hp.choice('dropout1', [0.2,0.3,0.1]),
    
    'batch_size' : hp.choice('batch_size', [128,256,512]),
    'nb_epochs' : hp.choice('nb_epochs', [50]),

    'optimizer':  hp.choice('optimizer', [ 'adam','adadelta']),   
    'activation': 'relu'    
    }

time: 182 ms


### Definir busqueda bayesiana

In [11]:
#Objective function that hyperopt will minimize
from timeit import default_timer as timer

def objective(params):
    
#     import ml_metrics

    
    start = timer()
    print ('Params testing: ', params)
    print ('\n ')
    
    model = Sequential()
    model.add(LSTM(params['units1'], input_shape=(1,x_train.shape[1]), return_sequences=True))
    model.add(Dropout(params['dropout1']))
    model.add(LSTM(params['units2'], return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='mean_squared_error', optimizer='adam')
    # compile the model
    model.compile(optimizer=params['optimizer'], loss='binary_crossentropy', metrics=['accuracy'])


    logdir = "Resultados\\" + exp_name +"\\logs\\model"

    tensor_board = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1, profile_batch = 100000000)

    
    #includes the call back object
    model.fit(reshape_x_train, y_train, epochs=params['nb_epochs'], batch_size=params['batch_size'],
              verbose = 0, validation_data=(reshape_x_test, y_test),callbacks=[tensor_board])
     
    #predict the test set 
    score, acc = model.evaluate(reshape_x_test, y_test, verbose=0)
    
    run_time = timer() - start
    
    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([-acc, params, score, run_time])
    of_connection.close()
    
    
    print('Test accuracy:', acc)
 
    return {'loss': -acc, 'status': STATUS_OK, 'train_time': run_time,}



time: 145 ms


### Almacenar resultados de cada iteración

In [12]:
from hyperopt import tpe

tpe_algorithm = tpe.suggest

time: 179 ms


In [13]:
import csv

from hyperopt import Trials

bayes_trials = Trials()

# File to save first results
out_file = folder + '/gbm_results.csv'
of_connection = open(out_file, 'w')

writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'score','time'])
of_connection.close()

time: 235 ms


### Lanzar optimización

In [14]:
# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = 10, trials = bayes_trials,
            verbose = 1, rstate= np.random.RandomState(50))


Params testing:                                                                                                                                                        
{'activation': 'relu', 'batch_size': 512, 'dropout1': 0.3, 'nb_epochs': 50, 'optimizer': 'adadelta', 'units1': 128, 'units2': 128}                                     
Test accuracy:                                                                                                                                                         
0.4914204                                                                                                                                                              
Params testing:                                                                                                                                                        
{'activation': 'relu', 'batch_size': 256, 'dropout1': 0.3, 'nb_epochs': 50, 'optimizer': 'adam', 'units1': 64, 'units2': 10}                                    

### Exportar bayesiana, por si quisiera retomar donde queda

In [15]:
import pickle
pickle.dump(bayes_trials, open(folder + '/trials.pkl', 'wb'))

time: 3 ms


### Leer mejores parametros

In [16]:
# Sort the trials with lowest loss (highest AUC) first
bayes_trials_results  = sorted(bayes_trials.results, key = lambda x: x['loss'])
bayes_trials_results [:2]

[{'loss': -0.49491578340530396,
  'status': 'ok',
  'train_time': 183.40774599999997},
 {'loss': -0.4918970465660095, 'status': 'ok', 'train_time': 166.3682008}]

time: 136 ms


In [17]:
results = pd.read_csv(folder + '/gbm_results.csv')

# Sort with best scores on top and reset index for slicing
results.sort_values('loss', ascending = True, inplace = True)
results.reset_index(inplace = True, drop = True)
results.head()

,loss,params,score,time
0,-0.494916,"{'activation': 'relu', 'batch_size': 256, 'dro...",0.761239,183.407746
1,-0.491897,"{'activation': 'relu', 'batch_size': 512, 'dro...",0.693182,166.368201
2,-0.491579,"{'activation': 'relu', 'batch_size': 256, 'dro...",0.693573,125.723013
3,-0.491579,"{'activation': 'relu', 'batch_size': 128, 'dro...",0.694035,295.828129
4,-0.491579,"{'activation': 'relu', 'batch_size': 128, 'dro...",0.693995,217.515321


time: 117 ms


In [18]:
import ast

# Convert from a string to a dictionary
ast.literal_eval(results.loc[0, 'params'])

{'activation': 'relu',
 'batch_size': 256,
 'dropout1': 0.3,
 'nb_epochs': 50,
 'optimizer': 'adam',
 'units1': 10,
 'units2': 512}

time: 125 ms


In [19]:
# Extract the ideal number of estimators and hyperparameters
best_bayes_params = ast.literal_eval(results.loc[0, 'params']).copy()
best_bayes_params

{'activation': 'relu',
 'batch_size': 256,
 'dropout1': 0.3,
 'nb_epochs': 50,
 'optimizer': 'adam',
 'units1': 10,
 'units2': 512}

time: 137 ms


### Definir datasets de testeo

In [20]:
# Selecciono la fecha para la cual hago el corte de train y test
training_end = pd.to_datetime("2014-12-31")
num_training = len(embeddings_average_individual[(embeddings_average_individual["Date"]) <= training_end])

time: 278 ms


In [21]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_average_individual

time: 159 ms


In [22]:
# Se separa en train y test
x_train = data.drop(["Top","Label", "Date"], axis=1)[:num_training]
x_test = data.drop(["Top",'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]


x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(len(x_train), 1, x_train.shape[1])
reshape_x_train.shape

x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(len(x_test), 1, x_train.shape[1])
reshape_x_test.shape

(9443, 1, 311)

time: 308 ms


In [23]:
model = Sequential()
model.add(LSTM(best_bayes_params['units1'], input_shape=(1,x_train.shape[1]), return_sequences=True))
model.add(Dropout(best_bayes_params['dropout1']))
model.add(LSTM(best_bayes_params['units2'], return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='mean_squared_error', optimizer='adam')
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


#model.compile(loss='mean_squared_error', optimizer='adam')
# compile the model
model.compile(loss='binary_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])


# define the checkpoint
filepath= ch_folder + "/word2vec-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

time: 1.64 s


In [24]:

logdir = "Resultados\\" + exp_name +"\\logs\\model"


tensor_board = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1, profile_batch = 100000000)


time: 2 ms


In [25]:
# fit the model
model.fit(reshape_x_train, y_train,
          epochs=best_bayes_params['nb_epochs'], 
          batch_size=best_bayes_params['batch_size'], callbacks=[tensor_board])


model.save(folder + '/keras_model.h5')
model.summary()

Train on 40208 samples
Epoch 1/50
40208/40208 [==============================] - 8s 199us/sample - loss: 0.6904 - accuracy: 0.5383
Epoch 2/50
40208/40208 [==============================] - 4s 93us/sample - loss: 0.6901 - accuracy: 0.5391
Epoch 3/50
40208/40208 [==============================] - 4s 91us/sample - loss: 0.6898 - accuracy: 0.5391
Epoch 4/50
40208/40208 [==============================] - 4s 95us/sample - loss: 0.6892 - accuracy: 0.5396
Epoch 5/50
40208/40208 [==============================] - 4s 93us/sample - loss: 0.6887 - accuracy: 0.5404
Epoch 6/50
40208/40208 [==============================] - 4s 92us/sample - loss: 0.6884 - accuracy: 0.5386
Epoch 7/50
40208/40208 [==============================] - 4s 99us/sample - loss: 0.6880 - accuracy: 0.5412
Epoch 8/50
40208/40208 [==============================] - 4s 93us/sample - loss: 0.6875 - accuracy: 0.5439
Epoch 9/50
40208/40208 [==============================] - 4s 92us/sample - loss: 0.6869 - accuracy: 0.5430
Epoch 10/50
4

In [26]:
# evaluate the model
loss, accuracy = model.evaluate(reshape_x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy))

Accuracy: 0.502383
time: 3.95 s


In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint()

In [ ]:
from time import sleep
sleep(60)

In [ ]:
from shutil import copyfile
copyfile('RNN_Model_Base_GPU.ipynb', folder + '/RNN_Model_Base.ipynb' )